In [1]:
import quandl
import pandas as pd
import numpy as np
import datetime
import pytrends as pt
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing, cross_validation, svm
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

%matplotlib inline

/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = quandl.get("WIKI/MSFT")

In [3]:
df.shape[0]

8076

In [5]:
Y1 = pd.DataFrame()
for i in range(252):
    k = df.shape[0]-252
    Y1[df.index[i+k]] = list(df.iloc[i+k-252:i+k]['Adj. Close'])
Y2 = pd.DataFrame()
for i in range(252):
    k = df.shape[0]-252
    Y2[df.index[i+k]] = list(df.iloc[i+k-5:i+k]['Adj. Close'])
Z = pd.DataFrame(columns=Y1.columns)
Z = Z.T
Z['M252'] = Y1.mean()
Z['S252'] = Y1.std()
Z['Sk252'] = Y1.skew()
Z['K252'] = Y1.kurt()
Z['M5'] = Y2.mean()
Z['S5'] = Y2.std()
Z['Sk5'] = Y2.skew()
Z['K5'] = Y2.kurt()
Z= Z.fillna(Z.mean())
Z.tail()

,M252,S252,Sk252,K252,M5,S5,Sk5,K5
2018-03-21,77.358864,9.553977,0.412976,-1.133319,93.730,0.714038,-0.065636,-2.029636
2018-03-22,77.472270,9.563211,0.402984,-1.151439,93.456,0.896119,0.423270,-2.167557
2018-03-23,77.577813,9.551753,0.393050,-1.169875,92.578,1.751648,-1.023844,2.255267
2018-03-26,77.669797,9.532069,0.382020,-1.182563,91.094,2.564748,-1.118355,-0.233106
2018-03-27,77.788596,9.545600,0.374272,-1.197794,91.272,2.745099,-0.963235,-0.541741


In [7]:
df[['Adj. Close']].shift(-1)
W = pd.merge(Z, df[['Adj. Close']].shift(-1),left_index=True,right_index=True)
W = W[:-1]

In [8]:
W.head()

,M252,S252,Sk252,K252,M5,S5,Sk5,K5,Adj. Close
2017-03-27,56.280720,5.014545,-0.093777,-1.155918,63.777681,0.331971,-2.066011,4.391245,64.255984
2017-03-28,56.331228,5.028673,-0.104637,-1.153629,63.811142,0.355213,-1.947067,3.933907,64.433133
2017-03-29,56.378025,5.047370,-0.114165,-1.154206,64.023721,0.153826,0.719431,0.970531,64.669332
2017-03-30,56.424232,5.067641,-0.122820,-1.155077,64.110328,0.236752,0.421419,-1.199486,64.816957
2017-03-31,56.470691,5.089391,-0.130517,-1.155463,64.275667,0.286508,0.386567,-1.028334,64.511866


In [9]:
X = W.drop(['Adj. Close'],1).fillna(0)
y = W[['Adj. Close']]

In [10]:
mm = MinMaxScaler()
mm.fit(X)
Xs = pd.DataFrame(mm.transform(X))

In [11]:
Xt, Xv, yt, yv = cross_validation.train_test_split(Xs, y, test_size = 0.3)
print Xt.shape

(175, 8)


In [12]:
model = MLPRegressor()

In [13]:
param_grid = dict(
activation = ['identity', 'logistic', 'tanh', 'relu'],
solver = ['lbfgs', 'sgd', 'adam'],
learning_rate = ['constant', 'invscaling', 'adaptive']
)

In [14]:
grid = GridSearchCV(cv = 3, 
                    estimator = model,
                    n_jobs = -1,
                    param_grid=param_grid,
)

In [15]:
grid.fit(Xs,y)

/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: M

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': ['constant', 'invscaling', 'adaptive'], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
model = grid.best_estimator_
model.fit(Xt,yt)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='invscaling',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [17]:
model.score(Xv,yv)

0.9776601721195477

In [18]:
forecast_prediction = model.predict(mm.transform(X[-1:]))
print(forecast_prediction)

[ 89.88504829]
